# <center> WizardCoder-15B-V1.0

#### Resources checkout

In [ ]:
!nvidia-smi

#### Donwnloads (run once)

In [ ]:
!pip3 install  -Uq transformers
!pip3 install  -Uq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip3 install  -Uq sentencepiece
!pip3 install  -Uq accelerate
!pip3 install  -Uq bitsandbytes

#### Imports

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

#### Loading model to VRAM

In [ ]:
BASE_MODEL_ID = 'WizardLM/WizardCoder-15B-V1.0'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID,
                                            device_map="auto", # {"":0} - load all wieghts to vram
                                            trust_remote_code=True,
                                            load_in_8bit=True)

#### Prompt

In [ ]:
# Here is your prompt
prompt = '''create Java classes according to this PlantUML diagram:

@startuml

entity Book {
id: long
name: string
language: string
yearOfPublishing: int
}

entity Author {
id: long
firstName: string
lastName: string
nationality: string
}

enum Genre {
SCIENCE_FICTION
FANTASY
HORROR
TECHNICAL_WRITING
EROTIC_FICTION
WESTERN
OTHER
}

Book "1..N" -o "1" Author: books
Book "1" o- "1..N" Genre: genres
Author "1" o- "1..N" Genre: genres
Book "1..N" o- "1" Author: author

@enduml'''

# Here is template that model can understand your prompt and make response 
prompt_template = f'''Below is an instruction that describes a task. Write a response that appropriately completes the request

### Instruction: {prompt}

### Response:'''

#### Inference

In [ ]:
# %timeit -r 5

generation_config = GenerationConfig(
    top_p=0.0,
    top_k=1,
    temperature=0.0,
    max_new_tokens=600,
    repetition_penalty=1.15,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id
)   

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoding = tokenizer(prompt_template, return_tensors="pt").to(device)

# with torch.cuda.amp.autocast(): ?????
# with torch.no_grad(): ????
# with torch.inference_mode(): # ????
outputs = model.generate(
    input_ids=encoding.input_ids,
    attention_mask=encoding.attention_mask,
    generation_config=generation_config,
    do_sampe = False,
    use_cache = True
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))